In [ ]:
import numpy as np
import math
import pickle
import os

from src.focus_lens_quadric_primitive import FocusLensQuadricPrimitive
from src.focus_doe_quadric_primitive import FocusDOEQuadricPrimitive
import src.utils_energy_distribution as en_util
import src.utils_optimization as opt_util
import src.utils_display as disp_util
import src.utils_files as files_util

dtype = np.double

In [ ]:
batch_folder_name = "quad_batches"
batch_file_default_name = "quad_batch"
batch_file_ext = ".ebv"

save_surface_file_name = 'data/16x16_surf.ebv'
load_surface_file_name = 'data/16x16_surf.ebv'

Create new dataset

1. Генерация лучей "rays" и точек "exit_points" в выходной плоскости. Лучи выбираются на апертуре радиусом "src_app_radius".

In [ ]:
num_samples_in_batch = int(512)
num_batches = 13
delete_folder_if_exists = False

exit_x_num = 16

exit_f_distance = 3e2
exit_x_size = 50

src_num_rays = int(4e4)
src_app_radius = 5
n_ref = 1.49
use_closest_surface = False

total_flux = 1

project_params = {'src_app_radius': src_app_radius, 'total_flux': total_flux, 'exit_x_size': exit_x_size,
                  'dtype': dtype}

In [ ]:
# create rays
nx = math.ceil(math.sqrt(4 / np.pi * src_num_rays))
nx = 2 * math.floor(nx/2)
r = np.linspace(-src_app_radius, src_app_radius, nx)
x_rays, y_rays = np.meshgrid(r, r)
good_rays = (x_rays ** 2 + y_rays ** 2) <= src_app_radius ** 2
src_num_rays = np.count_nonzero(good_rays)

rays = np.empty((src_num_rays, 3), dtype=dtype)
rays[:, 0] = x_rays[good_rays]
rays[:, 1] = y_rays[good_rays]
rays[:, 2] = total_flux / src_num_rays

# create exit plane coords
dl = exit_x_size / exit_x_num
l = np.linspace(-(exit_x_size + dl) / 2, (exit_x_size + dl) / 2, exit_x_num)
x_exit, y_exit = np.meshgrid(l, l)
exit_points = np.empty((exit_x_num, exit_x_num, 3), dtype=dtype)
exit_points[:, :, 0] = x_exit
exit_points[:, :, 1] = y_exit
exit_points[:, :, 2] = exit_f_distance

2. Инициализация примитива "QuadricPrimitive" используемой поверхности. Отвечает за трассировку, экспорт и обновление параметров.

In [ ]:
quadric_surface = FocusLensQuadricPrimitive(exit_points, exit_f_distance, n_ref, use_closest_surface)
# quadric_surface = FocusDOEQuadricPrimitive(exit_points, exit_f_distance, use_closest_surface)

In [ ]:
# Save descr file
quadric_surface.save(save_surface_file_name)

In [ ]:
# quadric_surface = FocusLensQuadricPrimitive.load(load_surface_file_name)
# quadric_surface = FocusDOEQuadricPrimitive.load(load_surface_file_name)
#
# with open(load_project_file_name, 'rb') as file:
#     project = pickle.load(file)
#
#     quadrics = project['quadrics']
#     distributions = project['design_logs']['optim_req_distrs']
#     rays = project['rays']

3. Подготовка к оптимизации - рассчет базовых значений параметров квадрик, которые будут референтными для остальных

In [ ]:
### Параметры для оптимизации 16х16:
### C фиксацией, плоский фронт, [1e-1, 1e-4]: (15к лучей, 5к итераций, err = 1.63),(25к лучей, 5к итераций, err = 0.92)
### Без фиксации, плоский фронт, [1e-1, 1e-4]: (15к лучей, 1.5к итераций, err = 1.24),(25к лучей, 1.5к итераций, err = 0.72), (40к лучей, 1.5к итераций, err = 0.51)
###
###


init_quad_value = 20
quadrics = init_quad_value

optim_start_step = 1e1
optim_end_step = 1e-4
optim_num_iters_base = int(1.5e3)

uniform_req_distr = np.ones((1, exit_x_num, exit_x_num), dtype=dtype) / (exit_x_num*exit_x_num)
fixed_quads = np.zeros((1, exit_x_num, exit_x_num), dtype=bool)
fixed_quads[0, int(np.ceil(exit_x_num/2)), int(np.ceil(exit_x_num/2))] = True

uniform_quadric, loss, time_total, quads_log = opt_util.run_oliker_optimization(quadric_surface, uniform_req_distr, rays,
                                                                                 quadrics=quadrics,
                                                                                 # fixed_quads=fixed_quads,
                                                                                 step=optim_start_step,
                                                                                 num_iters=optim_num_iters_base,
                                                                                 end_step=optim_end_step
                                                                                 )

In [ ]:
tracing_result, _ = quadric_surface.trace_rays_batch(uniform_quadric, rays)
rrmse = en_util.masked_rrmse_loss_batch(uniform_req_distr, tracing_result)
print(rrmse)
disp_util.display_random_pairs([uniform_req_distr], [tracing_result], titles=['Required image','Traced Quadrics'])
disp_util.display_2D_func(loss[0,:],title='Loss per iters')

4. Запуск основного цикла по батчам. На каждой итерации происходит:
        Инициализация требуемых распределений освещенности и начальных значений квадрик
        Оптимизация квадрик с помощью вызова статической функции "opt_util.run_oliker_optimization"
        Запись результатов в новый файл

In [ ]:
import shutil

# Prepare folder and file names for data export
if delete_folder_if_exists and os.path.isdir(batch_folder_name):
    shutil.rmtree(batch_folder_name)

if not os.path.isdir(batch_folder_name):
     os.mkdir(batch_folder_name)

filename_index = 1
with os.scandir(batch_folder_name) as it:
    for entry in it:
        if entry.is_file:
            base, ext = os.path.splitext(entry.name)
            if base.startswith(batch_file_default_name):
                base_parts = base.partition(batch_file_default_name+"_")
                cur_index = int(base_parts[2])
                filename_index = max(cur_index+1, filename_index)

In [ ]:
np_max = int(0.8 * exit_x_num ** 2)
np_min = int(0.2 * exit_x_num ** 2)

# optim_start_step = 1e1
# optim_end_step = 1e-4
optim_num_iters = int(0.8 * optim_num_iters_base)

In [ ]:
for k in range(num_batches):
    # Generate required distributions
    distributions = en_util.random_distribution(exit_x_num, np_min, np_max, total_flux=total_flux, batch_size=num_samples_in_batch, dtype=dtype)

    # Find center of each req_distr and freeze it quadrics with precalculated one on the uniform field
    inds = np.indices((exit_x_num, exit_x_num))
    fixed_quads = np.zeros(distributions.shape, dtype=bool)
    quadrics = np.zeros(distributions.shape, dtype=dtype)
    for i in range(num_samples_in_batch):
        good_vals = distributions[i] > 0

        y_inds = inds[0][good_vals]
        x_inds = inds[1][good_vals]

        inds_dist = (y_inds - np.mean(y_inds)) ** 2 + (x_inds - np.mean(x_inds)) ** 2
        center_ind = np.argmin(inds_dist)

        y_center = y_inds[center_ind]
        x_center = x_inds[center_ind]

        quadrics[i,:] = uniform_quadric[0,y_center,x_center]
        fixed_quads[i,y_center,x_center] = True


    # Optimize quadrics
    quadrics, loss, time_total, _ = opt_util.run_oliker_optimization(quadric_surface, distributions, rays,
                                                                             quadrics=quadrics,
                                                                             # fixed_quads=fixed_quads,
                                                                             step=optim_start_step,
                                                                             num_iters=optim_num_iters,
                                                                             end_step=optim_end_step
                                                                             )

    tracing_results, _ = quadric_surface.trace_rays_batch(quadrics, rays)
    rrmse = en_util.masked_rrmse_loss_batch(distributions, tracing_results)
    optim_params = np.tile([optim_start_step, optim_end_step, optim_num_iters], (num_samples_in_batch,1))

    new_file_name = batch_file_default_name + "_" + str(filename_index + k) + batch_file_ext
    new_file_name = os.path.join(batch_folder_name, new_file_name)
    files_util.save_quads(new_file_name, quadrics, distributions, rays, project_params,
                   loss=loss, time_total=time_total, rrmse=rrmse,
                   optim_params=optim_params, tracing_results=tracing_results)

    print(f'It{filename_index + k} RRMSE: Mean = {rrmse.mean()}; Max = {rrmse.max()}; Min = {rrmse.min()}')